In [1]:
import os
import time

# Self-code imports
import polytopes.segmentation_algorithms as algos
from polytopes.model.note import Note
from polytopes.model.chord import Chord
import polytopes.polytopical_costs as pc
import polytopes.data_manipulation as dm
import polytopes.segmentation_helper as sh
import polytopes.pattern_manip as pm
import polytopes.pattern_factory as pf

#Generic imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import copy

In [2]:
# Hyperparams
max_size = 47
min_size = 8
positive_segment_size_penalty = 0
negative_segment_size_penalty = 0.125
target_size = 32
positive_penalty = 2.25
negative_penalty = 3

# On a song

In [3]:
flow_file_path = "C:\\Users\\amarmore\\Desktop\\Annotations MIREX Quaero\\annotations_as_flow\\"
song = "001"
bag_of_chords = []
file = open(flow_file_path + song + ".flow", "r") 
for chord in file.readlines():
    bag_of_chords.append(chord.split("\n")[0])

In [4]:
frontiers_global, cost_global = algos.dynamic_minimization_guichaoua(bag_of_chords, global_antecedents = True)
frontiers, cost = algos.dynamic_minimization_guichaoua(bag_of_chords, global_antecedents = False)

print("My frontiers: {}".format(frontiers))
print("My frontiers, but global: {}".format(frontiers_global))
print("Cost: {}, and global: {}".format(cost, cost_global))

guichaoua_estimates = open(flow_file_path + "Guichaoua Estimates\\Experiments\\" + song + ".frontiers_estimated", "r")
tab = guichaoua_estimates.read().split(" ")
guichaoua_frontiers = [int(i) - 1 for i in tab if i != "\n"]
guichaoua_frontiers.append(frontiers[-1])
print("Guichaoua's frontiers: {}".format(guichaoua_frontiers))
print("Equality ? {}\n".format(guichaoua_frontiers == frontiers))

c:\users\amarmore\desktop\projects\phd main projects\on git\code\polytopes and relation\polytopes\segmentation_algorithms.py:398: UserWarning: No Polytope is available for this size of segment. Trying sequential cost instead.
  warnings.warn("No Polytope is available for this size of segment. Trying sequential cost instead.")


My frontiers: [0, 20, 52, 84, 112, 144, 176, 192, 220, 252, 284, 292, 332, 364, 396, 428, 462]
My frontiers, but global: [0, 20, 52, 84, 112, 144, 176, 192, 220, 252, 284, 292, 332, 364, 396, 428, 462]
Cost: 113.25, and global: 113.25
Guichaoua's frontiers: [0, 20, 52, 84, 112, 144, 176, 192, 220, 252, 284, 292, 332, 364, 396, 428, 462]
Equality ? True



In [5]:
string = ""
for i in frontiers:
    string += str(i + 1) + " "
file = open(flow_file_path + "Marmoret Estimates\\" + song + ".frontiers_estimated", "w")
file.write(string)
file.close()

In [6]:
#frontiers = [0, 20,52,84,112,144,176,192,220,252,284, 292, 332,364,396, 428, 462]
the_sum = 0
for start, end in zip(frontiers[:-1], frontiers[1:]):
    segment = bag_of_chords[start:end]
    #print(segment)
    size = len(segment)
    bag_of_patterns = sh.compute_patterns_with_global_antecedents_for_size(size)
    the_min = math.inf
    best_pattern = None
    for a_pattern in bag_of_patterns:
        cost = pc.guichaoua_cost_global_antecedents_successors(segment, a_pattern[0], a_pattern[3], a_pattern[4], a_pattern[5])
        cost += pc.irregularities_penalty_guichaoua(adding_code = a_pattern[1], deleting_code = a_pattern[2], positive_penalty = positive_penalty, negative_penalty = negative_penalty)
        if cost < the_min:
            best_pattern = a_pattern
            the_min = cost
    #print(best_pattern)
    the_min += sh.penalty_cost_guichaoua(size, target_size = 32, positive_segment_size_penalty = positive_segment_size_penalty, negative_segment_size_penalty = negative_segment_size_penalty)
    #print(pm.recursive_chords_on_pattern(best_pattern[0],segment))
    print("Cost: {} at {}".format(the_min, (start, end)))#, pm.recursive_chords_on_pattern(best_pattern[0],segment)))
    #print("local_cost: {}\n".format(local_costs[end - 1]))
    the_sum += the_min
print("Ze sum:{}".format(the_sum))

Cost: 7.75 at (0, 20)
Cost: 6 at (20, 52)
Cost: 5 at (52, 84)
Cost: 8.5 at (84, 112)
Cost: 4 at (112, 144)
Cost: 6 at (144, 176)
Cost: 7.0 at (176, 192)
Cost: 8.5 at (192, 220)
Cost: 4 at (220, 252)
Cost: 6 at (252, 284)
Cost: 6.0 at (284, 292)
Cost: 13.25 at (292, 332)
Cost: 7 at (332, 364)
Cost: 6 at (364, 396)
Cost: 8 at (396, 428)
Cost: 10.25 at (428, 462)
Ze sum:113.25


In [7]:
segment = bag_of_chords[268:304]
print(segment)
size = len(segment)
bag_of_patterns = sh.compute_patterns_with_global_antecedents_for_size(size)
the_min = math.inf
best_pattern = None
for a_pattern in bag_of_patterns[0:1]:
    cost = pc.guichaoua_cost_global_antecedents_successors(segment, a_pattern[0], a_pattern[3], a_pattern[4], a_pattern[5])
    #cost += pc.irregularities_penalty_guichaoua(adding_code = a_pattern[1], deleting_code = a_pattern[2], positive_penalty = positive_penalty, negative_penalty = negative_penalty)
    if cost < the_min:
        best_pattern = a_pattern
        the_min = cost
print(pf.apply_chords_on_pattern(best_pattern[0],segment))
print("Cost: {}".format(cost))

['Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'Eb', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'Ebm', 'Ebm', 'Ebm', 'Ebm']
[[[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['F#', 'F#'], ['Eb', 'Eb']]]], [[[['Abm', 'Abm'], ['Abm', 'Abm']], [['Abm', 'Abm'], ['E', 'Eb']]], [[['Ebm', 'Ebm'], ['Ebm', 'Ebm']], [[('Ebm', 'Ebm'), ('Ebm', 'Ab')], [('Ebm', 'Ebm'), ('Ebm', 'Ebm')]]]]]
Cost: 15


In [8]:
the_sum = 0
for start, end in zip(guichaoua_frontiers[:-1], guichaoua_frontiers[1:]):
    segment = bag_of_chords[start:end]
    size = len(segment)
    bag_of_patterns = sh.compute_patterns_with_antecedents_for_size(size)
    the_min = math.inf
    best_pattern = None
    for a_pattern in bag_of_patterns:
        cost = pc.guichaoua_cost(segment, a_pattern[0], a_pattern[3], a_pattern[4], a_pattern[5])
        cost += pc.irregularities_penalty_guichaoua(adding_code = a_pattern[1], deleting_code = a_pattern[2], positive_penalty = positive_penalty, negative_penalty = negative_penalty)
        if cost < the_min:
            best_pattern = a_pattern
            the_min = cost
    the_min += sh.penalty_cost_guichaoua(size, target_size = 32, positive_segment_size_penalty = positive_segment_size_penalty, negative_segment_size_penalty = negative_segment_size_penalty)
    #print(pm.recursive_chords_on_pattern(best_pattern[0],segment))
    print("Cost: {} at {}".format(the_min, (start, end)))
    the_sum += the_min
print("Ze sum:{}".format(the_sum))

Cost: 7.75 at (0, 20)
Cost: 6 at (20, 52)
Cost: 5 at (52, 84)
Cost: 8.5 at (84, 112)
Cost: 4 at (112, 144)
Cost: 6 at (144, 176)
Cost: 7.0 at (176, 192)
Cost: 8.5 at (192, 220)
Cost: 4 at (220, 252)
Cost: 6 at (252, 284)
Cost: 6.0 at (284, 292)
Cost: 13.25 at (292, 332)
Cost: 7 at (332, 364)
Cost: 6 at (364, 396)
Cost: 8 at (396, 428)
Cost: 10.25 at (428, 462)
Ze sum:113.25


In [9]:
segment = bag_of_chords[252:292]
#print(segment)
size = len(segment)
bag_of_patterns = sh.compute_patterns_with_antecedents_for_size(size)
the_min = math.inf
best_pattern = None
for a_pattern in bag_of_patterns:
    print(pf.recursive_chords_on_pattern(a_pattern[0],segment))
    cost = pc.guichaoua_cost(segment, a_pattern[0], a_pattern[3], a_pattern[4], a_pattern[5])
    print("Cost: {}".format(cost))#, pm.recursive_chords_on_pattern(best_pattern[0],segment)))
    cost += pc.irregularities_penalty_guichaoua(adding_code = a_pattern[1], deleting_code = a_pattern[2], positive_penalty = positive_penalty, negative_penalty = negative_penalty)
    print("Cost irreg': {}".format(cost))#, pm.recursive_chords_on_pattern(best_pattern[0],segment)))
    cost += sh.penalty_cost_guichaoua(size, target_size = 32, positive_segment_size_penalty = positive_segment_size_penalty, negative_segment_size_penalty = negative_segment_size_penalty)
    print("Cost penalty: {}".format(cost))#, pm.recursive_chords_on_pattern(best_pattern[0],segment)))


[[[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['B', 'B'], ['B', 'B']]]], [[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[[('E', 'E'), ('E', 'E')], [('F#', 'F#'), ('Eb', 'Eb')]], [[('Abm', 'Abm'), ('Abm', 'Abm')], [('Abm', 'Abm'), ('E', 'Eb')]]]]]
Cost: 10
Cost irreg': 12.25
Cost penalty: 12.25
[[[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['B', 'B'], ['B', 'B']]]], [[[['Eb', 'Eb'], ['Eb', 'Eb']], [[('Abm', 'Abm'), ('Abm', 'Abm')], [('E', 'E'), ('E', 'E')]]], [[['F#', 'F#'], ['Eb', 'Eb']], [[('Abm', 'Abm'), ('Abm', 'Abm')], [('Abm', 'Abm'), ('E', 'Eb')]]]]]
Cost: 11
Cost irreg': 13.25
Cost penalty: 13.25
[[[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['B', 'B'], ['B', 'B']]]], [[[['Eb', 'Eb'], [('Eb', 'Eb'), ('Abm', 'Abm')]], [['Abm', 'Abm'], [('E', 'E'), ('E', 'E')]]], [[['F#', 'F#'], [('Eb', 'Eb'), ('Abm', 'Abm')]], [